# Lab 7-2: Markov Chains - ENSO Phases

Download the data file for this lab, [ENSO_to2021.csv](https://mountain-hydrology-research-group.github.io/data-analysis/modules/data/ENSO_to2021.csv), which contains a record of the El Niño Southern Oscillation (ENSO) phase from 1900-2021.

You can read more about ENSO [here](https://www.weather.gov/mhx/ensowhat), and [here](https://www.climate.gov/enso).

---

Importing python packages you'll need for this lab:

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy import sparse
import matplotlib.pyplot as plt
%matplotlib inline

Load the data file

In [2]:
df = pd.read_csv('ENSO_to2021.csv', comment='#')
data = df['ENSO Phase']
# np.random.seed(1)
df.head(3)

,Water Year,ENSO Phase
0,1900,1
1,1901,2
2,1902,2


---
**A.** Using the time series of the phase of the El Niño Southern Oscillation (ENSO) from 1900-2021, create a lag-1 Markov model of the ENSO phase.

Observed Phases of ENSO:
 - 1: warm (El Niño)  
 - 2: neutral (ENSO neutral)  
 - 3: cool, (La Niña)  

Count transitions between each of the three ENSO phases using [scipy.sparse.csr_matrix()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html) and then [scipy.sparse.csr_matrix.todense()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.todense.html).

In [3]:
# count the transitions from each state to the next
# This counts the transitions from each state to the next and marks that count
S = sparse.csr_matrix((np.ones_like(data[:-1]), (data[:-1], data[1:])), dtype=float)
# convert transition counts to matrix form
# This converts those counts to matrix form
tm = S.todense()
print(tm)

[[ 0.  0.  0.  0.]
 [ 0. 11. 12. 17.]
 [ 0. 12. 15. 10.]
 [ 0. 16. 10. 18.]]


Normalize the transition matrix to get probabilities. This will create our lag-1 Markov Model.

In [4]:
tm_norm = tm / tm.sum(axis=1)
    
print(tm_norm) # This is our normalized transition matrix.

[[       nan        nan        nan        nan]
 [0.         0.275      0.3        0.425     ]
 [0.         0.32432432 0.40540541 0.27027027]
 [0.         0.36363636 0.22727273 0.40909091]]


<ipython-input-4-58e3ca30d8c7>:1: RuntimeWarning: invalid value encountered in true_divide
  tm_norm = tm / tm.sum(axis=1)


Compute cumulative sums along the rows, make sure these sum to 1. (We will use this cdf matrix below in a simulation of ENSO phases)

In [5]:
# We take the above probabilities of transitions, and turn them into discrete CDF's.
# These will allow us to map random numbers generated from a uniform distribution into 
# transitions that follow these probability rules.
tm_cdf = np.cumsum(tm_norm,1)

---
**B.** Using this Markov model and a random number generator, simulate 5,000 years of ENSO data.

In [6]:
# pick the number of years we want to simulate (5000)
n_years = 5000
# use a uniform random number for 5000 years
q = np.random.uniform(0,1,n_years); # uniformly distributed random numbers n_years long

# start off in state 2, neutral
initialstate = 2; # give it an initial state, doesn't really matter which

Nrand = np.zeros_like(q) # initialize an array of the proper size, with the initial state
Nrand[0] = initialstate;

# Now, just like we did when we created monte carlo simulations from empirical CDFs,
# we use our uniform random numbers to look up the next state in the transition matrix
for i in range(1,n_years):
    if q[i] <= tm_cdf[int(Nrand[i-1]),1]: #probability of transitioning from state i to 1
        Nrand[i] = 1;
    elif q[i] <= tm_cdf[int(Nrand[i-1]),2]: #transition from state i to 2
        Nrand[i] = 2;
    else:
        Nrand[i] = 3;

In [7]:
Nrand[-1]

2.0

---
**C.** Using this randomly generated data, answer the following questions. 

* According to the model, what is the probability that three warm ENSO years would occur in a row?
* What is the large-sample probability that three cool ENSO years would happen in a row?

(Try refreshing the numbers several times to increase the sample size if the condition never happens.) 

In [8]:
# And how many times did state 1 appear?
Test1 = [Nrand[0:-2], Nrand[1:-1], Nrand[2:]] # stack our data 3 times, shifting it to the right by 1 each time
Test1 = np.stack(Test1, axis=1)

G2 = np.where((np.max(Test1, axis=1) == 1) & (np.min(Test1, axis=1) == 1))
# if both the maximum and the minimum are 3, then we have 3 ones in our sequence

freqofthree1s = G2[0].size / Test1.shape[0]

print('Frequency of three warm ENSOs in a row = {}%'.format(100*np.round(freqofthree1s,3)))

Frequency of three warm ENSOs in a row = 2.3%


In [9]:
# And how many times did state 1 appear?
Test1 = [Nrand[0:-2], Nrand[1:-1], Nrand[2:]] # stack our data 3 times, shifting it to the right by 1 each time
Test1 = np.stack(Test1, axis=1)

G2 = np.where((np.max(Test1, axis=1) == 3) & (np.min(Test1, axis=1) == 3))
# if both the maximum and the minimum are 3, then we have 3 ones in our sequence

freqofthree1s = G2[0].size / Test1.shape[0]

print('Frequency of three cold ENSOs in a row = {}%'.format(100*np.round(freqofthree1s,3)))

Frequency of three cold ENSOs in a row = 5.5%


**According to the model, what is the probability that three warm ENSO years would occur in a row?**
The probability of three warm ENSO years in a row is approximately 2.3%

**What is the large-sample probability that three cool ENSO years would happen in a row? (Try refreshing the numbers several times to increase the sample size if the condition never happens.)**
The probability of three cool ENSO years in a row is approximately 5.5%